In [ ]:
%aiida

from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.base import Int, Str, Float
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from apps.reactions.replicawork import ReplicaWorkchain

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

# Generate Replicas

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    refresh_structure_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

    spin_ups = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("1")])
    spin_downs = ",".join([str(i) for i, k in enumerate(node.get_site_kindnames()) if k.endswith("2")])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

In [ ]:
from collections import OrderedDict

# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})
qb.order_by({Code: {'id': 'asc'}})

all_computers = OrderedDict()
all_computers['Please select a computer'] = False
for match in qb.all():
    code = match[1]
    label = match[0]
    all_computers[label] = code

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer", options=all_computers,
                            style=style, layout=layout)

int_machines = ipw.IntText(value=1,
                           description='# Nodes',
                           style=style, layout=layout)

text_cell = ipw.Text(placeholder='40 40 40',
                     description='Cell',
                     style=style, layout=layout)

text_fixed_atoms = ipw.Text(placeholder='1..10',
                            description='Fixed Atoms',
                            style=style, layout=layout)

text_colvar_targets = ipw.Text(placeholder='0.9 1.3 1.7 2.4',
                               description='Colvar Targets',
                               style=style, layout=layout)

text_colvar_type = ipw.Text(placeholder='DISTANCE',
                            description='Colvar Type',
                            style=style, layout=layout)

text_colvar_atoms = ipw.Text(placeholder='1 2 3',
                             description='Colvar Atoms',
                             style=style, layout=layout)

text_replica_name = ipw.Text(placeholder='Some cool name',
                             description='Replica Name',
                             style=style, layout=layout)

int_spring = ipw.IntText(description='Spring constant',
                         value=75.0,
                         style=style, layout=layout)


display(ipw.VBox([drop_computer, int_machines,
                  text_replica_name, text_cell, text_fixed_atoms,
                  text_colvar_targets, text_colvar_type, text_colvar_atoms, int_spring]))

In [ ]:
def on_submit(b):
    cp2k_code = drop_computer.value
    struct = struct_browser.results.value
    num_machines = Int(int_machines.value)
    cell = Str(text_cell.value)
    colvar_targets = Str(text_colvar_targets.value)
    colvar_type = Str(text_colvar_type.value)
    colvar_atoms = Str(text_colvar_atoms.value)
    replica_name = Str(text_replica_name.value)
    fixed_atoms = Str(text_fixed_atoms.value)
    spring = Float(int_spring.value)

    outputs = submit(ReplicaWorkchain,
                     cp2k_code=cp2k_code,
                     structure=struct,
                     num_machines=num_machines,
                     cell=cell,
                     colvar_targets=colvar_targets,
                     colvar_type=colvar_type,
                     colvar_atoms=colvar_atoms,
                     replica_name=replica_name,
                     fixed_atoms=fixed_atoms)
    print(outputs)
    the_workfunction = load_node(outputs.pid)
    the_workfunction.description = replica_name

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)

# Try atoms

In [ ]:
def on_preview(b):
    with output_preview:
        clear_output()
        t = deepcopy(struct_browser.results.value.get_ase())

        a_list = text_colvar_preview.value.split()
        for a in a_list:
            t[int(a)-1].symbol = 'O'

        if len(a_list) == 2:
            print 'Distance between the atoms:'
            d = t[int(a_list[0])-1].position-t[int(a_list[1])-1].position
            print d
            print np.linalg.norm(d)

        display(display_thumbnail(render_thumbnail(t)))
        #nglview.show_ase(t)
        
text_colvar_preview = ipw.Text(placeholder='1 2 3',
                               description='Colvar Atoms',
                               style=style, layout=layout)


btn_preview = ipw.Button(description='Try')
output_preview = ipw.Output()
btn_preview.on_click(on_preview)
display(text_colvar_preview, btn_preview, output_preview)